## Objective
Integration of LLM with Vector DB using Langchain

In [50]:
from langchain.chains import RetrievalQA, QAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import qdrant
from qdrant_client import QdrantClient
from langchain import OpenAI
from qdrant_client.http import models
from langchain.llms import HuggingFaceTextGenInference
embeddings_model_name = "all-MiniLM-L6-v2"

# Connecting to VLLM model server

In [51]:
openai_api_key = "llama/models/llama-2-7b-chat.ggmlv3.q4_0.bin"
openai_api_base = "http://34.126.87.255:32768/v1"
# openai_api_base = "http://35.188.99.103:8080/v1"
# Open_ai_llm = OpenAI(openai_api_key = openai_api_key,openai_api_base=openai_api_base, model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin")
Open_ai_llm = OpenAI(openai_api_key = openai_api_key,openai_api_base=openai_api_base, model = "meta-llama/Llama-2-7b-chat-hf",logit_bias=None)

In [52]:
Open_ai_llm.generate(["Hlo"])

LLMResult(generations=[[Generation(text='om provides a comprehensive range of services to help businesses and individuals create and manage their online presence. With a focus on custom design, development, and digital marketing solutions, Hloom helps clients establish and maintain a strong online presence that aligns with their brand values and goals.\nHloom’s comprehensive suite of services includes:\n1. Website Design and Development: Hloom creates custom, responsive websites that are optimized for user experience and search engine ranking.\n2. Digital Marketing: Hloom provides a range of digital marketing services, including SEO, PPC, social media marketing, and email marketing, to help clients reach their target audience and drive conversions.\n3. Branding and Identity: Hloom helps clients establish and maintain a strong brand identity through custom logo design, branding guidelines, and marketing materials.\n4. Content Creation: Hloom provides high-quality content creation servic

## Qdrant server

In [53]:
# client = QdrantClient(url="http://localhost",port=6333)
# collection_name = "VidColl"
client = QdrantClient(url="http://34.173.165.171",port=9333)
collection_name = "VideoCollection"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
qdrant_instance = qdrant.Qdrant(client, collection_name, embeddings=embeddings)

### Query parameters

In [54]:
query = "List product mentioned by Andrea Ali"
creater_filter = 'Ali Andreea'

In [55]:
retriever = qdrant_instance.as_retriever(search_kwargs={"k": 10
,"filter": models.Filter(
    must=[models.FieldCondition(key="metadata.creatorName", match=models.MatchAny(any=[creater_filter]))
        #  ,models.FieldCondition(key ="metadata.brandsMentioned", match=models.MatchAny(any=['nespresso']))
         ]
      )})

docs = retriever.get_relevant_documents(query)
# Giving sources in documents
for i,d in enumerate(docs):
    d.metadata['source'] = i


#### QA chain of Langchain + LLM  ---- QA with sources.

In [56]:
query_dic = {'question':query,
            'docs':  docs}
# print(query_dic)
qa_src = QAWithSourcesChain.from_chain_type(llm=Open_ai_llm, chain_type="stuff", return_source_documents = True)
res = qa_src(query_dic)
print("\n------------Results--------------\n",res)


Prompts-----
 [StringPromptValue(text='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES").\nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\n \n\nQUESTION: What creator has talked most about \'adidas\' in the past six months?\n=========\nContent: because like we\'ve had this discussion before right like there are there are lots of brands who\nSource: 7b53a9b84f8d269c192e3a7a-112\nContent: that in the empties. I like a nice clean pair of trainers. Adidas though. I don\'t I never:\nSource: 95f182c76a6e041efb53c08e-243\nContent: did not take my shoes off because of Adidas. I\'m wearing Adidas today. I think I\'m wearing. I don\'t know\nSource: a2d4f9c56b8e7e1d093d6f2c-278\nContent: women\'s pair of adidas I always have to wear men\'s this is probably why I like to take my shoes off\nSource: d7f6a8c3b5e2497e01538a69-456\n===

### RetrievalQA

In [57]:
qa = RetrievalQA.from_chain_type(llm=Open_ai_llm, chain_type="stuff", retriever=retriever)
result=qa(query)
print("\n\n---------Results from retrievalQA------->\n\n",result)


Prompts-----
 [StringPromptValue(text="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nOne product that I haven't talked about and I should have talked about it at the beginning of the video,\n\nAnd I'm also going to talk about each product that I think it's worth a mention in this video.\n\nThe next product would be, of course, you heard me talk about this product so often.\n\nThis product I love, love, love.\n\nI hope you enjoyed this video. If you have any questions about products or if you have any ideas about subjects you wouldn't want me to touch in my videos, please let me know in the comments.\n\nBut if you guys are not sure, please check out my description box where you will have every single product linked along with the colors that correspond to my skin.\n\nAre you going to stop talking about products so much?\n\nThey have amazing products, but by far my fav